#### **Import libraries**

In [1]:
# For working with files
import os
import re
import csv
import glob

# For manipulating data
import pandas as pd
import numpy as np

#### **Custom Functions**

##### **Build Dictionaries**

In [2]:
def load_csvs(working_dir):
    dataframes = {}
    filenames = []
    
    # Store all csv files as list
    files = glob.glob(working_dir + '/' + '/*.csv')
    
    # Find CSV files in working directory and reads each CSV file 
    for filepath in files:
        filename = os.path.splitext(os.path.basename(os.path.basename(filepath).split('/')[-1]))[0]
        filenames.append(filename)
        dataframes[filename] = pd.read_csv(filepath, encoding='latin-1')
    
    print(filenames)
    # Returns a dictionary containing all dataframes 
    return dataframes

In [3]:
def store_conferences(file):
    """
    Converts a CSV file with two columns (ConfAbbrev, Description) to a dictionary with ConfAbbrev as the key and Description as the value.

    Parameters:
    file (str): The path and name of the CSV file to convert.

    Returns:
    Returns a dictionary with ConfAbbrev as the key and Description as the value.

    Use:
    Use to get the Description (full name) for a conference using its abbreviation (eg. conferences['a_sun'])
    """

    # Create an empty dictionary to store the data
    conferences = {}

    # Open the csv file and read the contents into a list of dictionaries
    with open(file, 'r', encoding='latin-1') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        # Use a dictionary comprehension to create a dictionary from the csv data
        conferences = {row['ConfAbbrev']: row['Description'] for row in csv_reader}

    # Return dictionary
    return conferences

In [4]:
def store_spellings(file):
    """
    Converts a CSV file with two columns (TeamNameSpelling, TeamID) to a dictionary with TeamNameSpelling as the key and TeamID as the value.

    Parameters:
    file (str): The path and name of the CSV file to convert.

    Returns:
    Returns a dictionary with TeamNameSpelling as the key and TeamID as the value.

    Use:
    Use to get the TeamID for a team using an alternate spelling of the name (eg. mens_spellings['mt-st-marys'])
    """

    # Create an empty dictionary to store the data
    spellings = {}

    # Open the csv file and read the contents into a list of dictionaries
    with open(file, 'r', encoding='latin-1') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        # Use a dictionary comprehension to create a dictionary from the csv data
        spellings = {row['TeamNameSpelling']: row['TeamID'] for row in csv_reader}

    # Return dictionary
    return spellings

In [5]:
def store_team_conference(file):
    """
    Converts a CSV file with three columns (Season, TeamID, ConfAbbrev) to a nested dictionary with Season as the outer key, TeamID as the
    inner key, and ConfAbbrev as the value.

    Parameters:
    file (str): The path and name of the CSV file to convert.

    Returns:
    Returns a nested dictionary with Season as the outer key, TeamID as the inner key, and ConfAbbrev as the value.

    Use:
    Use to get the conference abbreviation for a team in a specific season (eg. mens_conferences['1985']['1449'])
    """

    # Create an empty nested dictionary
    conferences = {}

    # Open the CSV file and read the contents into a list of dictionaries
    with open(file, 'r', encoding='latin-1') as file:
        csv_reader = csv.DictReader(file)

        # Iterate over the rows in the CSV file
        for row in csv_reader:
            # Extract the year, id, and name from the current row
            year = row['Season']
            team_id = row['TeamID']
            abbrv = row['ConfAbbrev']

            # Create the outer dictionary if it doesn't exist
            if year not in conferences:
                conferences[year] = {}

            # Add the name to the inner dictionary with the id as the key
            conferences[year][team_id] = abbrv

    return conferences

In [6]:
def store_cities(file):
    """
    Converts a CSV file with three columns (CityID, City, State) to a dictionary with CityID as the key and both City, State as the value.

    Parameters:
    file (str): The path and name of the CSV file to convert.

    Returns:
    Returns a dictionary for City and State using CityID as the key.

    Use:
    Use to get the City and State for a game using the CityID (eg. city['4030'])
    """

    # Create an empty dictionary to store the data
    cities = {}

    # Open the csv file and read the contents into a list of dictionaries
    with open(file, 'r', encoding='latin-1') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        # Iterate over each row in the csv data
        for row in csv_reader:
            # Extract the relevant fields from the row
            city_id = row['CityID']
            city = row['City']
            state = row['State']

            # Add the data to the city_data dictionary
            cities[city_id] = f"{city}, {state}"

    # Return the dictionary
    return cities

In [7]:
def store_seasons(file):
    """
    Converts a CSV file with six columns (Season, DayZero, RegionW, RegionX, RegionY, RegionZ) to TWO dictionaries: 
    (1) with Season as the key and DayZero as the value; and 
    (2) with Season as the key and State as the value

    Parameters:
    file (str): The path and name of the CSV file to convert.

    Returns:
    Returns two dictionaries for City and State using CityID as the key.

    Use:
    Use to get the City and State for a game using the CityID (eg. city['4030'])
    """

    # Create five dictionaries with Season as the key and the other columns as the value
    dict_zero = {}
    dict_w = {}
    dict_x = {}
    dict_y = {}
    dict_z = {}

    with open(file, 'r', encoding='latin-1') as csvfile:
        reader = csv.reader(csvfile)
        #header = next(reader) # skip header row
        for row in reader:
            # Extract the values from the row
            season, day_zero, region_w, region_x, region_y, region_z = row
            
            # Populate the dictionaries
            dict_zero.setdefault(season, []).append(day_zero)
            dict_w.setdefault(season, []).append(region_w)
            dict_x.setdefault(season, []).append(region_x)
            dict_y.setdefault(season, []).append(region_y)
            dict_z.setdefault(season, []).append(region_z)

    # Return the dictionaries as a tuple
    return dict_zero, dict_w, dict_x, dict_y, dict_z

##### **Build Tables**

In [8]:
def build_teams(gender='mens', season=2023):
    # Create an list team ids for the specified season
    teamsm = []
    teamsw = []
   
    # Loop through mens team to get list from season
    for i in MTeams.loc[MTeams['LastD1Season'] == season]['TeamID']:
        teamsm.append(int(i))
        teamsw.append(int(i) + 2000)

    # Check if each team ID is present in the corresponding gender's team table
    if gender == 'womens':
        teamsw = [team for team in teamsw if team in WTeams['TeamID'].values]
    elif gender == 'mens':
        teamsm = [team for team in teamsm if team in MTeams['TeamID'].values]

    # Build women's team table
    if gender == 'womens':
        team = WTeams.loc[WTeams['TeamID'].isin(teamsw)].drop(['TeamName'], axis='columns')
        team['season'] = season
        team = team.iloc[:,[1,0]]
        team.rename(columns = {'season':'season', 'TeamID':'team_id'}, inplace = True)

    # Build men's team table
    elif gender == 'mens':
        team = MTeams.loc[MTeams['TeamID'].isin(teamsm)].drop(['TeamName', 'FirstD1Season'], axis='columns').iloc[:,[1,0]]
        team.rename(columns = {'LastD1Season':'season', 'TeamID':'team_id'}, inplace = True)
   
    else:
        pass

    return team

In [9]:
def get_mean_rank_by_system_season_team(df, season, team_id):
    """
    This function takes a dataframe, season, and team_id as inputs and returns a new dataframe with the mean rank
    (over all DayNum) by system within the specified Season and TeamID.
    """
    # Filter the dataframe for the specified season and team_id
    filtered_df = df[(df['Season'] == season) & (df['TeamID'] == team_id)]
    
    # Group the filtered dataframe by SystemName and calculate the mean rank
    grouped_df = filtered_df.groupby(['SystemName'])['OrdinalRank'].mean().round(0).reset_index()
    
    # Sort the grouped dataframe by mean rank
    sorted_df = grouped_df.sort_values(by=['OrdinalRank'], ascending=True)
    
    return sorted_df

In [10]:
def get_mean_rank_by_system_season_team(df, season, team_id):
    """
    This function takes a dataframe, season, team_id and systems as inputs and returns a new dataframe with the mean rank
    (over all DayNum) by system within the specified Season and TeamID.
    """
    systems = ['AP', 'BBT', 'BWE', 'D1A', 'DII', 'DOK', 'DUN', 'KPK', 'MAS', 'MOR', 
           'PGH', 'PIG', 'POM', 'RTP', 'SAG', 'TRK', 'TRP', 'USA', 'WIL', 'DES', 
           'LOG', 'DAV', 'FAS', 'FSH', 'NOL', 'RSL', 'SFX', 'SMN', 'BUR', 'COL', 
           'CRO', 'DCI', 'DDB', 'HAS', 'INP', 'RT', 'RTH', 'SEL', 'TPR', 'WOL', 
           'ARG', 'CNG', 'KPI', 'MCL', 'WLK', 'DC2', 'JRT', 'RPI', 'SP', 'STH', 
           'BIH', 'SPR', '7OT', 'DOL', 'LMC', 'SPW', 'EBP', 'ESR', 'FMG', 'PRR', 
           'REW', 'SMS', 'DC', 'KRA', 'WOB', 'HKS', 'MUZ', 'OCT', 'SGR', 'ZAM', 
           'ADE', 'JNG', 'CRW', 'HKB', 'PMC', 'YAG']

    # Filter the dataframe for the specified season and team_id
    filtered_df = df[(df['Season'] == season) & (df['TeamID'] == team_id)]
    
    # Group the filtered dataframe by SystemName and calculate the mean rank
    grouped_df = filtered_df.groupby(['SystemName'])['OrdinalRank'].mean().round(0).reset_index()
    
    # Create a dictionary with SystemName as key and mean rank as value
    system_dict = dict(zip(grouped_df['SystemName'], grouped_df['OrdinalRank']))
    
    # Create a new dictionary with each SystemName as a key and its mean rank as the value, 
    # with missing values set to NaN
    output_dict = {system: system_dict.get(system, float('nan')) for system in systems}
    
    # Add the season and team_id to the output dictionary
    output_dict['Season'] = season
    output_dict['TeamID'] = team_id
    
    # Convert the output dictionary to a dataframe
    output_df = pd.DataFrame([output_dict])
    
    return output_df

In [11]:
def get_rating_pivot(df):
    """
    This function takes a dataframe as input and returns a new dataframe with the mean rank
    (over all DayNum) by system within each Season and TeamID.
    """
    # Group the dataframe by Season, TeamID, and SystemName and calculate the mean rank
    grouped_df = df.groupby(['Season', 'TeamID', 'SystemName'])['OrdinalRank'].mean().round(0).reset_index()
    
    # Pivot the grouped dataframe to create a new dataframe with SystemNames as columns
    pivoted_df = grouped_df.pivot(index=['Season', 'TeamID'], columns='SystemName', values='OrdinalRank')
    
    return pivoted_df

#### **Load Dictionaries**

In [12]:
# Create a dictionary to store alternate team name spellings
mens_spellings = store_spellings('dictionaries/MTeamSpellings.csv') # use as mens_spellings[team]
womens_spellings = store_spellings('dictionaries/WTeamSpellings.csv') # use as womens_spellings[team]

# Create a dictionary to store alternate team name spellings
mens_conferences = store_team_conference('dictionaries/MTeamConferences.csv') # use as mens_conferences[year][team_id]
womens_conferences = store_team_conference('dictionaries/WTeamConferences.csv') # use as womens_conferences[year][team_id]

# Create a dictionary to store conference full names
conferences = store_conferences('dictionaries/Conferences.csv') # use as conferences[abbrv]

# Create a dictionary to store cities and states
cities = store_cities('dictionaries/Cities.csv') # use as cities[city_id]

# Create a dictionary to store day zero and each region by season year
mens_d0, m_w, m_x, m_y, m_z = store_seasons('dictionaries/MSeasons.csv') # use as mens_d0[season], m_w[season], etc
womens_d0, w_w, w_x, w_y, m_z = store_seasons('dictionaries/WSeasons.csv') # use as womens_d0[season], w_w[season], etc

#### **Load Files**

In [13]:
MTeams = pd.read_csv('data/MTeams.csv')
WTeams = pd.read_csv('data/WTeams.csv')

In [14]:
MGameCities = pd.read_csv('data/MGameCities.csv')
WGameCities = pd.read_csv('data/WGameCities.csv')

In [15]:
Ordinals = pd.read_csv('data/MMasseyOrdinals.csv')

In [ ]:
systems = ['AP', 'BBT', 'BWE', 'D1A', 'DII', 'DOK', 'DUN', 'KPK', 'MAS', 'MOR', 'PGH', 'PIG', 'POM', 'RTP', 'SAG', 'TRK', 'TRP', 'USA', 'WIL', 'DES', 'LOG', 'DAV', 'FAS', 'FSH', 'NOL', 'RSL', 'SFX', 'SMN', 'BUR', 'COL', 'CRO', 'DCI', 'DDB', 'HAS', 'INP', 'RT', 'RTH', 'SEL', 'TPR', 'WOL', 'ARG', 'CNG', 'KPI', 'MCL', 'WLK', 'DC2', 'JRT', 'RPI', 'SP', 'STH', 'BIH', 'SPR', '7OT', 'DOL', 'LMC', 'SPW', 'EBP', 'ESR', 'FMG', 'PRR', 'REW', 'SMS', 'DC', 'KRA', 'WOB', 'HKS', 'MUZ', 'OCT', 'SGR', 'ZAM', 'ADE', 'JNG', 'CRW', 'HKB', 'PMC', 'YAG']

# Group the data by multiple columns
season_ratings = Ordinals.groupby('Season')
grouped = Ordinals.groupby(['Season', 'TeamID'])

result = pd.DataFrame(columns = ['system', 'mean', 'count'])

for sys in systems:
    if sys in Ordinals.columns:
        # Calculate the mean of a column for each group
        mean_values = (grouped['OrdinalRank'].mean() / season_ratings['TeamID'].sum()).round(3)
        
        # Calculate the count of rows in each group
        count_values = grouped.size()
        
        # Combine the two Series into a DataFrame
        #result = pd.concat([sys, mean_values, count_values], axis=1)
        
        # append rows to an empty DataFrame
        result = result.append({'system' : sys, 'mean' : mean_values, 'count' : count_values}, ignore_index = True)

# Display the condensed table
print(result)

In [ ]:
def get_team_ratings(season, teamid):

    systems = ['AP', 'BBT', 'BWE', 'D1A', 'DII', 'DOK', 'DUN', 'KPK', 'MAS', 'MOR', 
               'PGH', 'PIG', 'POM', 'RTP', 'SAG', 'TRK', 'TRP', 'USA', 'WIL', 'DES', 
               'LOG', 'DAV', 'FAS', 'FSH', 'NOL', 'RSL', 'SFX', 'SMN', 'BUR', 'COL', 
               'CRO', 'DCI', 'DDB', 'HAS', 'INP', 'RT', 'RTH', 'SEL', 'TPR', 'WOL', 
               'ARG', 'CNG', 'KPI', 'MCL', 'WLK', 'DC2', 'JRT', 'RPI', 'SP', 'STH', 
               'BIH', 'SPR', '7OT', 'DOL', 'LMC', 'SPW', 'EBP', 'ESR', 'FMG', 'PRR', 
               'REW', 'SMS', 'DC', 'KRA', 'WOB', 'HKS', 'MUZ', 'OCT', 'SGR', 'ZAM', 
               'ADE', 'JNG', 'CRW', 'HKB', 'PMC', 'YAG']

    # Group the data by multiple columns
    season_ratings = Ordinals.groupby('Season')
    grouped = Ordinals.groupby(['Season', 'TeamID'])

    result = pd.DataFrame(columns=['season', 'system', 'mean', 'count'])

    for sys in systems:
        if sys in grouped['SystemName'].unique():
            # Filter the grouped DataFrame to only include rows with the current system
            system_grouped = grouped[grouped['SystemName'] == sys]
            
            # Reset the index of the grouped DataFrame to include TeamID as a regular column
            system_grouped = system_grouped.reset_index()

            # Calculate the mean of a column for each group
            mean_values = (system_grouped.groupby('Season')['OrdinalRank'].mean() / 
                           season_ratings['TeamID'].sum()).round(3)

            # Calculate the count of rows in each group
            count_values = system_grouped.groupby('Season').size()

            # Combine the two Series into a DataFrame
            system_result = pd.concat([mean_values, count_values], axis=1).reset_index()

            # Add the system name to the DataFrame
            system_result['system'] = sys

            # Rename the columns of the system result DataFrame
            system_result.columns = ['season', 'system', 'mean', 'count']

            # Append the system result DataFrame to the overall result DataFrame
            result = result.append(system_result, ignore_index=True)

    # Filter the result DataFrame to only include the desired season and team ID
    result = result[(result['season'] == season) & (result['system'] == sys)]

    # Reset the index of the result DataFrame
    result = result.reset_index(drop=True)

    return result

#### **Build Tables**

In [16]:
mens = build_teams('mens', 2023)
#mens.head()

In [17]:
womens = build_teams('womens', 2023)
#womens.head()

In [ ]:
ratings = get_rating_pivot(Ordinals).reset_index()
ratings.rename_axis(index=None);

In [ ]:
ratings_grouped = ratings.groupby('Season')
season_rating = ratings_grouped.get_group(2023)
pd.set_option('display.max_rows', season_rating.shape[0]+1)
season_rating.isnull().sum()

#### **No longer need**

In [ ]:
def pivot_table(df):
    # Group the dataframe by the id column and aggregate the names into a list
    grouped = df.groupby('TeamID')['TeamNameSpelling'].agg(list)
    
    # Convert the grouped data to a pivot table, where each id is a separate row and all the names that correspond to that id are in separate columns
    pivot_table = pd.DataFrame(grouped.values.tolist(), index=grouped.index)

    # Rename the columns
    pivot_table.columns = [f'Column {i+1}' for i in range(pivot_table.shape[1])]
    
    # Reset the index
    pivot_table = pivot_table.reset_index()
    
    # Print the pivot table
    return pivot_table